#  Установка библиотек

In [2]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline



                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit', 'chembl_structure_pipeline']

conda-forge/linux-64     Using cache
conda-forge/noarch     

# Импорт библиотек

In [4]:
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors
from matplotlib import pyplot as plt
%matplotlib inline
import json
from collections import defaultdict
from multiprocessing.pool import Pool
from pathlib import Path
from typing import Dict, List, Optional
import os
from rdkit import Chem
from rdkit.Chem.MolStandardize.tautomer import TautomerCanonicalizer
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
import torch.optim as optim


# Загрузка данных

In [5]:
!wget https://raw.githubusercontent.com/kostyastrong/HSE_nlp_mol_properties/master/data/lipophilicity.csv

--2021-05-09 21:49:33--  https://www.dropbox.com/s/dsrl00tj4zjmqbz/Lipophilicity.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dsrl00tj4zjmqbz/Lipophilicity.csv [following]
--2021-05-09 21:49:33--  https://www.dropbox.com/s/raw/dsrl00tj4zjmqbz/Lipophilicity.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc96385079ea9be1808d03b5d24.dl.dropboxusercontent.com/cd/0/inline/BOJi8PHi4cL6o9RSCUx3csA_1F3ysHPm0MZFbr1EbKm9eA2BBgGSq1-cZO4r35C5pAlMhmAm5y4z13zpEqAD47toNgfGPWdod4YYKg9IvYjpW3TJeJvJ1EJSnI-2pYYDANahqIOxKIrzK5m_lmC39KNf/file# [following]
--2021-05-09 21:49:34--  https://ucc96385079ea9be1808d03b5d24.dl.dropboxusercontent.com/cd/0/inline/BOJi8PHi4cL6o9RSCUx3csA_1F3ysHPm0MZFbr1EbKm9eA2BBgGSq1-cZO4r35C5pAlMhmAm5y4

# Предобработка данных

1. Сделайте стандартизацию SMILES в датасете (используйте код с TautomerCanonicalizer)

2. Проверьте, появились ли дубликаты в обработанных SMILES. Удалите их при помощи метода drop_duplicates()



In [6]:
data = pd.read_csv('lipophilicity.csv')

In [7]:
data.head(5)

,CMPD_CHEMBLID,exp,smiles
0,CHEMBL596271,3.54,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14
1,CHEMBL1951080,-1.18,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...
2,CHEMBL1771,3.69,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl
3,CHEMBL234951,3.37,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...
4,CHEMBL565079,3.10,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...


In [8]:
from typing import Dict, List, Optional
import os
from rdkit import Chem
from rdkit.Chem.MolStandardize.tautomer import TautomerCanonicalizer
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
import torch.optim as optim


class DatasetsHolder:
    @staticmethod
    def read_datasets(inp_folder_path):
        Dataframe = pd.read_csv(inp_folder_path)
        return Dataframe


class StandardizeDatasets:
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        m = Chem.MolFromSmiles(smi)
        m = Chem.MolToSmiles(m)
        return m

    def standardize(self, inp_path: Path, out_path: Path):
        smth = DatasetsHolder()
        data = smth.read_datasets(inp_path)
        data['st_smiles'] = data['smiles'].apply(self.standardize_smiles)
        data.to_csv(out_path)


class StandardizeTautomers(StandardizeDatasets):
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        tmp = TautomerCanonicalizer()
        m = Chem.MolFromSmiles(smi)
        m = tmp.canonicalize(m)
        return Chem.MolToSmiles(m)



In [9]:
'''with Pool(10) as pool:
    data['standartize_smiles'] = list(
                        tqdm(pool.imap(StandardizeTautomers.standardize_smiles, data.smiles), total=data.shape[0])
                    )'''

"with Pool(10) as pool:\n    data['standartize_smiles'] = list(\n                        tqdm(pool.imap(StandardizeTautomers.standardize_smiles, data.smiles), total=data.shape[0])\n                    )"

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


В данном ноутбуке мы будем работать с молекулами как со строками - с последовательностью символов SMILES.

Для того, чтобы нейросеть могла "проглотить" текстовые данные их нужно перевести в набор чисел.

1. Разделить строки SMILES на "токены". Токен - отдельная единица текста, можно воспринимать его как признак.

2. Сопоставить токенам порядковый номер в словаре.


Токенизация - процесс поиска определенных наборов символов в строке. Оптимально такой поиск совершать при помощи [регулярных выражений](https://docs.python.org/3/howto/regex.html).

**Задание**: 
1. Посмотрите на SMILES в вашем датасете. Подумайте, какие паттерны можно выделить из этих строк? (какие химические элементы присутствуют в молекулах, какие спец символы используются)
2. Придумайте pattern регулярного выражения, который сможет найти нужные токены. (подсказка - несколько паттернов указываются через символ  |)
3. Примените функцию токенизации ко всем smiles в датасете

In [11]:
data = pd.read_csv('/content/drive/MyDrive/HSE_data/norm_smiles.csv') #переведем smiles в норм вид

In [12]:
data.head(5)

,Unnamed: 0,CMPD_CHEMBLID,exp,smiles,standartize_smiles
0,0,CHEMBL596271,3.54,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,Cn1c(CN2CCN(c3ccc(Cl)cc3)CC2)nc2ccccc21
1,1,CHEMBL1951080,-1.18,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,COc1cc(OC)c(S(=O)(=O)N2c3ccccc3CCC2C)cc1NC(=O)...
2,2,CHEMBL1771,3.69,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,COC(=O)C(c1ccccc1Cl)N1CCc2sccc2C1
3,3,CHEMBL234951,3.37,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,O=C(NC1Cc2ccccc2N(C[C@@H](O)CO)C1=O)c1cc2cc(Cl...
4,4,CHEMBL565079,3.10,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,Cc1cccc(CC(NC(=O)c2cc(C(C)(C)C)nn2C)C(=O)NCC#N)c1


In [13]:
lst = ['C', 'n', '1', 'c', '(', 'N', '2', ')', '3', 'l', '4', '\t', '5', 'O', 'S', '=', '6', 's', '7', '[', '@', 'H', ']', '8', '#', '9', '-', '0', 'o', 'F', 'B', 'r', '+', '\\', '/', 'P', 'I', 'i', 'e', '.', '%']
str = ""
for i in lst:
    str = str + '|' + i
str

'|C|n|1|c|(|N|2|)|3|l|4|\t|5|O|S|=|6|s|7|[|@|H|]|8|#|9|-|0|o|F|B|r|+|\\|/|P|I|i|e|.|%'

In [14]:
class data_preprocessor:
  def __init__(self):
    self.symbols = []
    self.seq_length = -1
    self.seq_lengths = []


  def preprocess_data(self, df):
      self.tokenization(df)
      self.create_vocab()
      X = []
      for x in df.tokens:
          x = x.split(' ')
          x = self.seq2ind(x)
          x_eq = self.pad_input(x)
          X.append(x_eq)
      y = df.exp
      
      return X, y


  def tokenization(self, df):
    """
    Tokenize all SMILES in df dataframe
    """
    df['tokens'] = df['standartize_smiles'].apply(self.smiles_tokenization)
    self.seq_length = max(self.seq_lengths) #wtf
    return df

  def smiles_tokenization(self, smi: str):
    """
    Tokenize a SMILES molecule or reaction
    """
    # https://docs.python.org/3/library/re.html
    import re
    pattern = "Cl?|Br?|I|Se?|H|O|N|P|F|cl?|br?|i|se?|h|o|n|p|f|[0-9]|#|@|\(|\)|\[|\]|\-|%|\/|\\\\|\.|=|\+"
    regex = re.compile(pattern)
    
    tokens = regex.findall(smi)
    self.seq_lengths.append(len(tokens))
    self.symbols += tokens
    self.symbols = list(set(self.symbols))
    #print(smi)
    #print("".join(tokens))
    assert smi == ''.join(tokens)
    return ' '.join(tokens)

  def create_vocab(self):
    """
    Create vocabulary of the symbols
    """
    self.token2idx = {o:i for i,o in enumerate(self.symbols)}
    self.idx2token = {i:o for i,o in enumerate(self.symbols)}

  def seq2ind(self, seq: list):
    return [self.token2idx[x] for x in seq] #map of tokens

  def pad_input(self, seq: list):
    features = np.zeros((self.seq_length),dtype=int)
    features[-len(seq):] = np.array(seq)[:self.seq_length]
    return features
  

In [15]:
st = data_preprocessor()
X, y = st.preprocess_data(data)


# Делим данные на тренировочную и тестовую выборки

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [18]:
 np.array(y_train.values)[:10]

array([ 3.78,  0.87,  3.14,  3.9 ,  1.01,  2.7 ,  2.63,  1.5 , -0.06,
        0.76])

# Создаем загрузчики данных для Pytorch


  * Преобразуйте данные в numpy array
  * сделайте torch.tensor при помощи torch.from_numpy
  * создайте датасет torch.utils.data.TensorDataset
  * сделайте из него DataLoader


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape)


train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))

test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

batch_size = 400

train_loader = DataLoader(train_data, batch_size)

test_loader = DataLoader(test_data, batch_size)

(2940, 265)


In [20]:
print(y_train)

[3.78 0.87 3.14 ... 3.6  0.2  3.8 ]


In [21]:
iter(train_loader).next()

[tensor([[ 0,  0,  0,  ...,  1,  1,  9],
         [ 0,  0,  0,  ...,  9, 15,  5],
         [ 0,  0,  0,  ..., 11, 11,  9],
         ...,
         [ 0,  0,  0,  ..., 18,  1, 25],
         [ 0,  0,  0,  ...,  5, 18,  5],
         [ 0,  0,  0,  ..., 11,  9, 25]]),
 tensor([ 3.7800,  0.8700,  3.1400,  3.9000,  1.0100,  2.7000,  2.6300,  1.5000,
         -0.0600,  0.7600,  1.5600,  2.6800,  2.4000, -1.0300, -0.6500,  3.3300,
          0.6800,  1.3100,  3.1000,  0.5100,  3.8000,  0.6400,  1.9700,  2.4000,
          2.8000,  1.3000,  3.0200,  2.6000,  2.8600, -0.8700,  2.7000,  3.8000,
          3.0600,  4.0000, -0.3700,  1.9000, -0.5100,  1.1800,  2.1400,  1.2100,
          3.6000,  1.5000,  3.4000,  2.9000,  0.9700, -0.4000,  2.2100,  3.0800,
          3.6000,  2.4100,  0.2300,  1.7300,  2.2800,  2.1000,  3.0100,  2.6900,
          2.5800,  2.5000,  2.6000,  0.9100,  3.5000,  2.4100,  3.8600,  2.2300,
          3.6600,  1.4000,  3.3200,  1.1800,  0.7800,  1.7400,  2.8000,  3.7000,
         

In [22]:
for now in train_loader:
    print(type(now))
    print(len(now))
    print(now)
    break

<class 'list'>
2
[tensor([[ 0,  0,  0,  ...,  1,  1,  9],
        [ 0,  0,  0,  ...,  9, 15,  5],
        [ 0,  0,  0,  ..., 11, 11,  9],
        ...,
        [ 0,  0,  0,  ..., 18,  1, 25],
        [ 0,  0,  0,  ...,  5, 18,  5],
        [ 0,  0,  0,  ..., 11,  9, 25]]), tensor([ 3.7800,  0.8700,  3.1400,  3.9000,  1.0100,  2.7000,  2.6300,  1.5000,
        -0.0600,  0.7600,  1.5600,  2.6800,  2.4000, -1.0300, -0.6500,  3.3300,
         0.6800,  1.3100,  3.1000,  0.5100,  3.8000,  0.6400,  1.9700,  2.4000,
         2.8000,  1.3000,  3.0200,  2.6000,  2.8600, -0.8700,  2.7000,  3.8000,
         3.0600,  4.0000, -0.3700,  1.9000, -0.5100,  1.1800,  2.1400,  1.2100,
         3.6000,  1.5000,  3.4000,  2.9000,  0.9700, -0.4000,  2.2100,  3.0800,
         3.6000,  2.4100,  0.2300,  1.7300,  2.2800,  2.1000,  3.0100,  2.6900,
         2.5800,  2.5000,  2.6000,  0.9100,  3.5000,  2.4100,  3.8600,  2.2300,
         3.6600,  1.4000,  3.3200,  1.1800,  0.7800,  1.7400,  2.8000,  3.7000,
       

# Построение модели

## Сверточная нейросеть

In [23]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [24]:
class  CharCNN(nn.Module):
    def __init__(self, seq_len: int):
        super(CharCNN, self).__init__()

        self.embedding = nn.Embedding(len(st.token2idx) + 1, 400)



        self.conv1 = nn.Sequential(
            nn.Conv1d(seq_len, 128, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )

        self.fc1 = nn.Linear(16768, 1)

       
    def forward(self, x):
        x = self.embedding(x)
        x = self.conv1(x)
       
        # collapse
        x = x.view(x.size(0), -1)
        # linear layer
        x = self.fc1(x)

        
        return x

In [25]:
model = CharCNN(st.seq_length)
model = model.to(device)

import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), 
                       lr=0.001
                       )

In [26]:
from tqdm import tqdm
for it_now, labels in tqdm(train_loader):
    
    optimizer.zero_grad()

    it_now = it_now.to(device)
    labels = labels.to(device).view(-1,1)

    output = model(it_now.long())


    loss = criterion(output.float(), labels.float())

    loss.backward()
    
    optimizer.step()
    
    

100%|██████████| 8/8 [00:36<00:00,  4.56s/it]


In [27]:
print(type(labels))

<class 'torch.Tensor'>


**TO-DO**
1. Создайте optimizer, criterion
2. Переместите модель на GPU
3. Добавьте процедуру обучения

## RNN

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

## LSTM

![](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

Почитать - https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Посмотреть - https://stepik.org/course/54098/syllabus (Языковые модели и генерация текста) 

In [28]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__() #
        self.output_size = output_size
        self.n_layers = n_layers #?
        self.hidden_dim = hidden_dim #?
        
        # Сначала создаем слой, который будет тренироваться для 
        # перевода исходных последовательностей в вектора
        # nn.Embedding

        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim=embedding_dim)

        # Добавляем LSTM слой
        # nn.LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = drop_prob)

        # Добавляем DropOut между слоями нейронки
        self.dropout = nn.Dropout(drop_prob)

        # Линейный слой для вывода предсказания 
        self.fc = nn.Linear(512, output_size)


        
    def forward(self, x):#, hidden):
        batch_size = x.size(0)
        x = x.long()


        # создаем эмбеддинг
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    


In [29]:
vocab_size = len(st.token2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

**TO-DO**
1. Создайте optimizer, criterion
2. Переместите модель на GPU
3. Добавьте процедуру обучения


In [30]:
batch_size

400

In [31]:
len(train_loader)

8

In [32]:
model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model = model.to(device)

import torch.optim as optim
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), 
                       lr=0.001
                       )

train_loss = []
val_loss = []
val_acc = []

for epoch in range(10):
    running_loss = 0

    for it_now, labels in tqdm(train_loader):
        optimizer.zero_grad() #обнуление производных
        it_now = it_now.to(device)
        labels = labels.to(device) #все на гпу
        output = model(it_now.long())[0]
        '''print(len(output), len(labels))
        print(type(output[0]), type(labels[0]))
        print(output, labels, sep='\n')'''
        loss = criterion(output.float(), labels.float())
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        
    print(f"Epoch {epoch} - Training loss: {running_loss/len(train_loader)}")
    train_loss.append(running_loss)    

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 0 - Training loss: 2.462368741631508


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 - Training loss: 1.5060921609401703


  0%|          | 0/8 [00:00<?, ?it/s]

...

Epoch 7 - Training loss: 1.4253544509410858


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 8 - Training loss: 1.4186868965625763


100%|██████████| 8/8 [06:22<00:00, 47.75s/it]

Epoch 9 - Training loss: 1.4342459291219711


# Улучшения

1. (Неожиданная) аугментация данных.

Если у вас на руках есть небольшое количество данных (как у нас), то можно искусственно увеличить их количество. 

В данном случае можно воспользоваться методом, который мы рассматривали для уменьшения дубликатов в датасете. 

Для cоздания дополнительных SMILES в данных используйте `Chem.MolToSmiles и Chem.MolFromSmiles` с флагом `doRandom`.

2. Углубить модель (больше lstm слоев)

3. Заменить lstm на bidirectional lstm (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

4. Подобрать гиперпараметры (руками или GridSearch). embedding_dim, hidden_dim, количество LSTM слоев

5. Сделать кросс-валидацию

6. Добавить EarlyStopping

7. Попробовать добавить свойства молекул из бейзлайна как параллельный энкодер
Пример:
https://github.com/NU-CUCIS/CheMixNet

8. Использовать one-hot-encoding для кодирования последовательности текста


In [33]:
smi = 'COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl'
random_equivalent_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smi), doRandom=True)

In [34]:
random_equivalent_smiles

'COC([C@@H](N1CCc2c(ccs2)C1)c1c(cccc1)Cl)=O'